# **Digit warrior: beat the machine!**

This project was imagined and implemented by Alexiane Maillard, Business Administration student at the University of St.Gallen.

## Concept of the project
This project is a Python project where a small game was implemented. 
The game is called *Digit warrior: beat the machine* and is about a contest between the user and a machine. The user is shown an image of a handwritten digit (between 0 and 9, coming from the scikit-learn *digits* dataset) and must guess what number is written. Against him/her is a machine, a k-Nearest-Neighbors model, which has been trained to predict the number shown in the image.


## Main components of the project
1. `machine_creation()` function: In this function, a supervised ML model for digit prediction is created. The k-Nearest-Neighbor was used due to the satisfying model accuracy it provided (around 96%). The hyperparameter k was set to 3 after tuning (hyperparameter tuning done with a for-loop and several values for k, then deleted in the project below to only keep the final value, this to ensure smooth code in 1 cell), because it delivered the most satisfying model accuracy while not being too small (which would represent an overfitting danger). Finally, this function returns the digits dataset, the trained model and the model accuracy. The first two returned values are used in the `play_guess_number_game(dataset, knn_model)` function, the model accuracy is used in the game explanations from the `main()` function.
2. `user_input_btw_0_9`function: since the user must enter a number between 0 and 9 (included) when he plays the game, this function was defined to make sure that the input is valid, i.e. that it is an integer (or can be converted into an integer without causing a ValueError) between 0 and 9. As long as the user input is not valid, the user is asked to enter a new input. This function returns the user input, certified valid. The returned input is used in the `play_guess_number_game(dataset, knn_model)` function.
3. `play_guess_number_game(dataset, knn_model)` function: this function computes the actual game. It takes two arguments, both coming from the returned values of the `machine_creation()`funciton. As long as the user wants to play, it randomly selects and shows him a handwritten digit as an image to the user. On one side, it uses the user input computed in the previous function. On the other side, it uses the digit label predicted by the model (defined in the first function) and compares it with the user input. Then, it prints who was right and adjusts the user and the machine's scores accordingly. It prints both scores and asks if the user wants to play again (if yes, the while-loop of the function continues).
4. `main()` function: this function is the "display" function. It shows the game title and explanations and calls the previously defined functions when required. This function is then called following Python best practices.


ChatGPT has been of some help for this project. It is mentioned in the code what ChatGPT was used for.

In [7]:
# Various imports for model creation and game implementation
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
import numpy as np
import matplotlib.pyplot as plt
import random


# Model creation (this part was done with no help from ChatGPT)
def machine_creation():
    # Load the dataset
    digits = datasets.load_digits()

    # Create the kNN model
    ## Scale the data for better results of the model
    scaler = StandardScaler()
    data_scaled = scaler.fit_transform(digits.data)
    ## Split the data into train, test and validation datasets (random state for reproducibility, stratify to ensure balanced proportions of each label in both sub-datasets)
    x_train, x_test, y_train, y_test = train_test_split(data_scaled, digits.target, random_state=42, stratify=digits.target)

    ## Create the model with hyperparameter k, fit the model and do predictions
    k = 3 # Several hyperparameters were tried and this hyperparameter was manually selected for its good accuracy (around 96%)
    model = KNeighborsClassifier(n_neighbors=k)
    model.fit(x_train, y_train)
    y_pred = model.predict(x_test)
    accuracy = accuracy_score(y_pred, y_test) # Compute model accuracy

    return digits, model, accuracy


# Ask for user input and check if input is an integer between 0 and 9 (included, otherwise ask again until it is)
def user_input_btw_0_9():
    while True: #executed as long as input is not an int between 0 and 9
        try:
          # Ask for input, try to convert it into integer
          user_input = int(input("Please enter an integer: "))
          # Check if input is between 0 and 9: if not, ask the user to give an input until this condition is met
          if 0 <= user_input <=9:
             return user_input # returned value is surely an integer between 0 and 9
          else:
            print('The entered number is not between 0 and 9 (included). Please enter a valid number: ')
        except ValueError: #executed as long as input is not an int
          print('The entered value is not an integer. Please enter an integer: ')


# Game implementation: compare user and machine guesses and declare the winner
# ChatGPT was used for various parts of this function (especially for commenting)
def play_guess_number_game(dataset, knn_model):
    # Store user and machine scores in two variables
    user_score = 0
    machine_score = 0

    while True:

        # Pick a random image from the dataset digits
        index = random.randint(0, len(dataset.data) - 1)
        image = dataset.data[index]
        actual_label = dataset.target[index]

        # Display the image
        plt.imshow(image.reshape(8,8), cmap='gray') # ChatGPT used for this
        plt.title('What number is this?')
        plt.axis('off') # ChatGPT used for this
        plt.show()

        # Get user guess
        user_guess = user_input_btw_0_9()

        # Get model prediction, i.e. machine guess
        model_prediction = knn_model.predict([image])[0]

        # Compare guesses and actual label
        print(f"Your guess: {user_guess}")
        print(f"Machine's prediction: {model_prediction}")
        print(f"Actual number: {actual_label}")

        # Check if the user's guess was correct
        if user_guess == actual_label:
            print("Congratulations! You were right!")
        else:
            print("Wrong guess!")

        # Check if the user beat the machine and adapt their score accordingly
        if user_guess == actual_label and model_prediction != actual_label:
            user_score += 1
            print("Congratulations, you beat the machine! Well done!")
        elif model_prediction == actual_label and user_guess != actual_label:
            machine_score += 1
            print("The machine was right, better luck next time!")
        elif user_guess == model_prediction == actual_label:
            user_score += 1
            machine_score += 1
            print("Both you and the machine guessed correctly!")
        else:
            print("Neither you nor the machine guessed it right.")

        # Print the user's and machine's scores as a table framed by '-' characters
        print('-'*27)
        print(f'{"Your score":^11}|{"Machine score":^15}')
        print(f'{user_score:^11}|{machine_score:^15}')
        print('-'*27)

        # Ask if the user wants to play again
        play_again = input('Ready to challenge the machine again? Type the letter y to play, anything else to flee: ')
        if play_again.lower() != 'y':
            print('Okay, see you soon!')
            break

# Create main function, to combine all functions previously defined
def main():
  # Load the digits dataset, create the kNN model and retrieve its accuracy
  digits, model, model_accuracy = machine_creation()
  # Show game title and explanations
  print('DIGIT WARRIOR: BEAT THE MACHINE!\n\n')
  print(f'''Welcome to the DIGIT WARRIOR game. In this game, you have to guess what number is handwritten on a given image.
Against you? A machine trained to beat you by predicting what the number on the image is. The machine guesses correctly {model_accuracy*100:.2f}% of the time.''')
  play_or_not = input('Ready to challenge the machine? Type the letter y to play, anything else to flee: ')
  if play_or_not.lower() != 'y':
    print('Okay, see you soon!')
  else:
    play_guess_number_game(digits, model)

# Execute the program
if __name__ == "__main__":
    main()


DIGIT WARRIOR: BEAT THE MACHINE!


Welcome to the DIGIT WARRIOR game. In this game, you have to guess what number is handwritten on a given image. 
Against you? A machine trained to beat you by predicting what the number on the image is. The machine guesses correctly 96.67% of the time.
Ready to challenge the machine? Type the letter y to play, anything else to flee: h
Okay, see you soon!
